# ENSO como modelo conceptual mecanicista

Este notebook explora un modelo conceptual mecanicista de baja dimensión del fenómeno ENSO (El Niño–Southern Oscillation), formulado como un sistema dinámico simple.

El objetivo no es reproducir ENSO de forma realista, sino analizar cómo puede emerger una variabilidad interanual organizada a partir de la interacción entre un número reducido de procesos dominantes, representados mediante variables agregadas y acoplamientos explícitos.

El modelo utilizado pertenece a la familia de modelos conceptuales mecanicistas discutidos en la parte teórica de la asignatura.


## Objetivos

Al finalizar este notebook, el estudiante debería ser capaz de:

- Interpretar ENSO como un fenómeno climático que puede analizarse de forma aislada mediante un modelo conceptual mecanicista.
- Identificar el papel de acoplamientos rápidos y de variables con escalas temporales largas en la aparición de variabilidad organizada.
- Relacionar un diagrama cualitativo de interacciones con un sistema mínimo de ecuaciones diferenciales.
- Analizar el comportamiento de un sistema dinámico determinista sin recurrir a variabilidad en los forzamientos externos.


## Cómo debe utilizarse este notebook

En esta actividad, el estudiante deberá:

- ejecutar simulaciones ya definidas,
- modificar parámetros concretos del modelo,
- analizar e interpretar resultados gráficos.

No se espera que el estudiante:
- desarrolle código desde cero,
- realice cálculos analíticos,
- compare los resultados de forma cuantitativa con datos observacionales.


## Alcance y limitaciones

El modelo analizado en este notebook es deliberadamente simple. Está diseñado para capturar la estructura dinámica esencial de ENSO utilizando un número muy reducido de variables.

Como consecuencia:

- no se representan patrones espaciales,
- muchos procesos físicos se agrupan en términos simples,
- los valores numéricos no deben interpretarse como realistas.

El énfasis se sitúa exclusivamente en la comprensión del **comportamiento dinámico** que se deriva de la estructura del modelo.


## ENSO como sistema dinámico

ENSO puede interpretarse como el resultado de interacciones entre el océano tropical y la atmósfera que operan en escalas temporales diferentes.

En los modelos conceptuales mecanicistas, estas interacciones se representan mediante un conjunto reducido de variables agregadas y acoplamientos explícitos, que en conjunto definen un sistema dinámico.

Antes de introducir ecuaciones o código, comenzamos con una descripción cualitativa de estas interacciones.


## Diagrama conceptual de interacciones

El siguiente diagrama resume, a nivel conceptual, las interacciones dominantes implicadas en ENSO.

En él se destacan:
- el acoplamiento entre anomalías de temperatura superficial del océano y la circulación atmosférica,
- la presencia de un componente oceánico cuya evolución se produce en escalas temporales más largas,
- la estructura de retroalimentaciones que vincula ambos componentes.

![Diagrama conceptual de ENSO](../figures/ENSO_conceptual_diagram.png)


## Del diagrama conceptual al modelo matemático

El diagrama conceptual presentado en la sección anterior puede traducirse en un modelo matemático mínimo mediante un sistema de ecuaciones diferenciales ordinarias de baja dimensión.

En este enfoque, cada variable representa una magnitud agregada asociada a un componente clave del sistema océano–atmósfera, y cada término de las ecuaciones corresponde a una interacción identificada previamente a nivel conceptual.

El objetivo de este modelo no es describir procesos físicos elementales, sino representar de forma explícita la estructura dinámica esencial del fenómeno, manteniendo el número de variables y parámetros tan reducido como sea posible.


## Variables y ecuaciones del modelo

El modelo se formula en términos de dos variables dinámicas:

- $T(t)$: anomalía de la temperatura superficial del océano en el Pacífico ecuatorial (variable asociada a ENSO).
- $h(t)$: variable asociada al contenido de calor oceánico o a la profundidad efectiva de la termoclina, cuya evolución se produce en escalas temporales más largas.

La dinámica del sistema se describe mediante el siguiente sistema de ecuaciones diferenciales ordinarias:

$
\frac{dT}{dt} = a_{11}\,T + a_{12}\,h
$

$
\frac{dh}{dt} = a_{21}\,T + a_{22}\,h
$

donde los coeficientes $a_{ij}$ representan, de forma agregada, los acoplamientos y amortiguamientos entre las variables del sistema.


## Interpretación cualitativa del modelo

Cada término del sistema anterior tiene una interpretación cualitativa directa en relación con el diagrama conceptual:

- Los términos proporcionales a $T$ y $h$ en cada ecuación representan amortiguamientos y acoplamientos agregados entre el océano y la atmósfera.
- Los coeficientes cruzados ($a_{12}$ y $a_{21}$) describen la interacción entre ambas variables, responsable de la retroalimentación dinámica del sistema.
- La presencia de una variable cuya evolución es más lenta que la otra introduce desfases temporales en la respuesta del sistema, que resultan esenciales para la aparición de variabilidad organizada.

No se espera que el estudiante resuelva este sistema de forma analítica ni que realice un análisis matemático formal. El interés se centra exclusivamente en comprender cómo la estructura del modelo condiciona su comportamiento dinámico.


In [1]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt


ModuleNotFoundError: No module named 'numpy'

In [ ]:
# Parámetros del modelo (tiempo en meses)
# Nota: estos valores están pensados para exploración conceptual, no para ajuste cuantitativo a datos.

params = {
    "a11": -0.076,   # amortiguamiento efectivo de T (mes^-1)
    "a12":  0.0236,  # acoplamiento h -> T (mes^-1, h en "unidades" del modelo)
    "a21": -1.25,    # acoplamiento T -> h (mes^-1)
    "a22": -0.008    # amortiguamiento efectivo de h (mes^-1)
}


In [2]:
# Horizonte temporal (meses)
years = 80
t0 = 0.0
t1 = 12.0 * years

# Puntos donde queremos la salida (para graficar de forma estable)
dt_out = 0.25  # meses (salida cada ~7.5 días)
t_eval = np.arange(t0, t1 + dt_out, dt_out)

# Condiciones iniciales (puedes ajustarlas si quieres explorar sensibilidad)
T0 = 0.5   # K (anomalía inicial)
h0 = 0.0   # unidades del modelo
y0 = [T0, h0]


NameError: name 'np' is not defined

In [ ]:
sol = solve_ivp(
    fun=lambda t, y: recharge_oscillator_rhs(t, y, params),
    t_span=(t0, t1),
    y0=y0,
    t_eval=t_eval,
    method="RK45",
    rtol=1e-8,
    atol=1e-10
)

if not sol.success:
    raise RuntimeError("La integración ha fallado: " + sol.message)

t = sol.t
T = sol.y[0]
h = sol.y[1]


plt.figure(figsize=(5, 5))
plt.plot(T, h, lw=1.2)
plt.xlabel("T(t) (anomalía SST)")
plt.ylabel("h(t) (variable oceánica lenta)")
plt.title("Diagrama de fase (T–h)")
plt.grid(True, alpha=0.3)
plt.show()


## Preguntas de interpretación (simulación determinista)

1. Describe el tipo de comportamiento dinámico observado en $T(t)$ y $h(t)$ (oscilatorio amortiguado, sostenido, etc.).
2. ¿Se observa un desfase entre $T(t)$ y $h(t)$? ¿Cómo se aprecia en la figura temporal y en el diagrama de fase?
3. Cambia ligeramente un parámetro (por ejemplo, $a_{11}$ o $a_{22}$) y describe qué rasgos cambian y cuáles se mantienen.


## Preguntas conceptuales

Antes de continuar, reflexiona sobre las siguientes cuestiones:

- ¿Qué componente del sistema responde más rápidamente?
- ¿Qué componente evoluciona de forma más lenta?
- ¿Qué interacción introduce un desfase temporal en la respuesta del sistema?
- ¿Por qué este sistema puede mostrar variabilidad organizada incluso si los forzamientos externos permanecen constantes?

Estas preguntas deben responderse de forma cualitativa, a partir del diagrama anterior.
